# Python Configuration

pip install pyodbc openai pandas ipython tabulate

## MS SQL Server Library Config
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18

### optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools18
echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
source ~/.bashrc

### optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

### optional: kerberos library for debian-slim distributions
sudo apt-get install -y libgssapi-krb5-2

In [9]:
import pyodbc  
from openai import AzureOpenAI, OpenAIError  
import openai  
import json  
import pickle  
import random  
import pandas as pd  
from IPython.display import Markdown, display  
import time   


In [10]:
# Configuration  
openai_gpt_api_base =  "https://<redacted>.openai.azure.com/"
openai_gpt_api_key =  "<redacted>"
openai_gpt_api_version =   "2024-02-15-preview"
openai_gpt_model =  "gpt-4o"
openai_embedding_api_base = "https://<redacted>.openai.azure.com/"
openai_embedding_api_key = "<redacted>"
openai_embedding_api_version =  "2024-02-15-preview"
openai_embedding_model = "text-embedding-ada-002"
server = '<redacted>.database.windows.net'
database = '<redacted>'
username = '<redacted>'
password = '<redacted>'

openai_temperature = 0.0  
max_tokens = 4096 

In [11]:
# Function to generate table description using OpenAI  
def generate_oai_table_desc(content, openai_gpt_api_base, openai_gpt_api_key, openai_gpt_api_version, openai_gpt_model):  
    max_attempts = 6  
    max_backoff = 60  
    system_prompt = """  
    You are an intelligent assistant that helps create a description for a SQL Server table.   
    You will be provided with details such as the table name, columns and column details.  
    Do not create new column names or tables.  
    Please output this description in markdown format.  
    """  
    user_prompt = content  
    gpt_client = AzureOpenAI(  
        api_version=openai_gpt_api_version,  
        azure_endpoint=openai_gpt_api_base,  
        api_key=openai_gpt_api_key  
    )  
  
    counter = 0  
    incremental_backoff = 1  # seconds to wait on throttling - this will be incremental backoff  
    while counter < max_attempts:  
        try:  
            response = gpt_client.chat.completions.create(  
                model=openai_gpt_model,  
                messages=[  
                    {"role": "system", "content": system_prompt},  
                    {"role": "user", "content": user_prompt}  
                ],  
                temperature=openai_temperature,  
                max_tokens=max_tokens,  
                top_p=0.95,  
                frequency_penalty=0,  
                presence_penalty=0,  
                stop=None,  
                stream=False  
            )  
            return response.choices[0].message.content  
        except openai.APIError as ex:  
            if str(ex.code) == "429":  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                print('Waiting to retry after', incremental_backoff, 'seconds...')  
                counter += 1  
                time.sleep(incremental_backoff)  
            elif str(ex.code) == "DeploymentNotFound":  
                print('Error: Deployment not found')  
                return 'Error: Deployment not found'  
            elif 'Error code: 40' in str(ex):  
                print('Error: ' + str(ex))  
                return 'Error:' + str(ex)  
            elif 'Connection error' in str(ex):  
                print('Error: Connection error')  
                return 'Error: Connection error'  
            elif str(ex.code) == "content_filter":  
                print('Content Filter Error', ex.code)  
                return "Error: Content could not be extracted due to Azure OpenAI content filter." + ex.code  
            else:  
                print('API Error:', ex)  
                print('API Error Code:', ex.code)  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                counter += 1  
                time.sleep(incremental_backoff)  
        except Exception as ex:  
            counter += 1  
            print('Error - Retry count:', counter, ex)  
            return ""  
  
# Function to generate SQL query using OpenAI  
def generate_query(question, schema_details, previous_response, openai_gpt_api_base, openai_gpt_api_key, openai_gpt_api_version, openai_gpt_model):  
    max_attempts = 6  
    max_backoff = 60  
    system_prompt = """  
    You are an intelligent assistant that helps create a SQL Server query that retrieves data relating to a user's question.  
    You 'may' be provided with some "additional_data" which may help you to create this query.  
    Ensure your queries use both the owner and table names (for example dbo.Sales).  
    If you need more details from the user to create a valid query, please ask for these details, otherwise only return the SQL Query in your response.  
    Use only the schema details provided by the user and NEVER make up or guess table or column names.  
    For filtering using WHERE statements, never guess or assume the value to use for the WHERE value.  
    For example, if the question asks to show the data where state is WA. Do not assume the filter value is "WA".  
    If you need more information, please request it.  
    Unless you need more details, always respond with a SQL Query only as your response.  
    Before sending back a query, make sure everything needed to execute the query exists in the schema provided by the user.  
    """  
    user_prompt = "Please help me create a SQL Server query that retrieves data to answer this question: " + question  
    if previous_response:  
        user_prompt += 'additional_data: ' + previous_response + '\n\n'  
    user_prompt += "Here are the schema details for this database:\n" + schema_details  
  
    gpt_client = AzureOpenAI(  
        api_version=openai_gpt_api_version,  
        azure_endpoint=openai_gpt_api_base,  
        api_key=openai_gpt_api_key  
    )  
  
    counter = 0  
    incremental_backoff = 1  # seconds to wait on throttling - this will be incremental backoff  
    while counter < max_attempts:  
        try:  
            response = gpt_client.chat.completions.create(  
                model=openai_gpt_model,  
                messages=[  
                    {"role": "system", "content": system_prompt},  
                    {"role": "user", "content": user_prompt}  
                ],  
                temperature=openai_temperature,  
                max_tokens=max_tokens,  
                top_p=0.95,  
                frequency_penalty=0,  
                presence_penalty=0,  
                stop=None,  
                stream=False  
            )  
            return response.choices[0].message.content  
        except openai.APIError as ex:  
            if str(ex.code) == "429":  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                print('Waiting to retry after', incremental_backoff, 'seconds...')  
                counter += 1  
                time.sleep(incremental_backoff)  
            elif str(ex.code) == "DeploymentNotFound":  
                print('Error: Deployment not found')  
                return 'Error: Deployment not found'  
            elif 'Error code: 40' in str(ex):  
                print('Error: ' + str(ex))  
                return 'Error:' + str(ex)  
            elif 'Connection error' in str(ex):  
                print('Error: Connection error')  
                return 'Error: Connection error'  
            elif str(ex.code) == "content_filter":  
                print('Content Filter Error', ex.code)  
                return "Error: Content could not be extracted due to Azure OpenAI content filter." + ex.code  
            else:  
                print('API Error:', ex)  
                print('API Error Code:', ex.code)  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                counter += 1  
                time.sleep(incremental_backoff)  
        except Exception as ex:  
            counter += 1  
            print('Error - Retry count:', counter, ex)  
            return ""  
  
# Function to generate text answer using OpenAI  
def generate_text_answer(question, sql_query_response, openai_gpt_api_base, openai_gpt_api_key, openai_gpt_api_version, openai_gpt_model):  
    max_attempts = 6  
    max_backoff = 60  
    system_prompt = """  
    You are an intelligent assistant that helps take a user's question and a markdown table and tries to create a textual response to the user's question based on the data from the markdown table.  
    You can assume that the data included does answer the question.  
    Only use the information in the table to try to answer the question and do not make anything up.  
    """  
    user_prompt = 'Question: ' + question + '\n'  
    user_prompt += 'Answer: ' + sql_query_response + '\n'  
    gpt_client = AzureOpenAI(  
        api_version=openai_gpt_api_version,  
        azure_endpoint=openai_gpt_api_base,  
        api_key=openai_gpt_api_key  
    )  
  
    counter = 0  
    incremental_backoff = 1  # seconds to wait on throttling - this will be incremental backoff  
    while counter < max_attempts:  
        try:  
            response = gpt_client.chat.completions.create(  
                model=openai_gpt_model,  
                messages=[  
                    {"role": "system", "content": system_prompt},  
                    {"role": "user", "content": user_prompt}  
                ],  
                temperature=openai_temperature,  
                max_tokens=max_tokens,  
                top_p=0.95,  
                frequency_penalty=0,  
                presence_penalty=0,  
                stop=None,  
                stream=False  
            )  
            return response.choices[0].message.content  
        except openai.APIError as ex:  
            if str(ex.code) == "429":  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                print('Waiting to retry after', incremental_backoff, 'seconds...')  
                counter += 1  
                time.sleep(incremental_backoff)  
            elif str(ex.code) == "DeploymentNotFound":  
                print('Error: Deployment not found')  
                return 'Error: Deployment not found'  
            elif 'Error code: 40' in str(ex):  
                print('Error: ' + str(ex))  
                return 'Error:' + str(ex)  
            elif 'Connection error' in str(ex):  
                print('Error: Connection error')  
                return 'Error: Connection error'  
            elif str(ex.code) == "content_filter":  
                print('Content Filter Error', ex.code)  
                return "Error: Content could not be extracted due to Azure OpenAI content filter." + ex.code  
            else:  
                print('API Error:', ex)  
                print('API Error Code:', ex.code)  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                counter += 1  
                time.sleep(incremental_backoff)  
        except Exception as ex:  
            counter += 1  
            print('Error - Retry count:', counter, ex)  
            return ""  
  
# Function to generate vectors for text  
def generate_embedding(text, openai_embedding_api_version, openai_embedding_api_base, openai_embedding_api_key, openai_embeddings_model):  
    max_attempts = 6  
    max_backoff = 60  
    if text is None:  
        return None  
  
    client = AzureOpenAI(  
        api_version=openai_embedding_api_version,  
        azure_endpoint=openai_embedding_api_base,  
        api_key=openai_embedding_api_key  
    )  
  
    counter = 0  
    incremental_backoff = 1  # seconds to wait on throttling - this will be incremental backoff  
    while counter < max_attempts:  
        try:  
            response = client.embeddings.create(  
                input=text,  
                model=openai_embeddings_model  
            )  
            return json.loads(response.model_dump_json())["data"][0]['embedding']  
        except OpenAIError as ex:  
            if str(ex.code) == "429":  
                print('OpenAI Throttling Error - Waiting to retry after', incremental_backoff, 'seconds...')  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                counter += 1  
                time.sleep(incremental_backoff)  
            elif str(ex.code) == "DeploymentNotFound":  
                print('Error: Deployment not found')  
                return 'Error: Deployment not found'  
            elif 'Error code: 40' in str(ex):  
                print('Error: ' + str(ex))  
                return 'Error:' + str(ex)  
            elif 'Connection error' in str(ex):  
                print('Error: Connection error')  
                return 'Error: Connection error'  
            elif str(ex.code) == "content_filter":  
                print('Content Filter Error', ex.code)  
                return "Error: Content could not be extracted due to Azure OpenAI content filter." + ex.code  
            else:  
                print('API Error:', ex)  
                print('API Error Code:', ex.code)  
                incremental_backoff = min(max_backoff, incremental_backoff * 1.5)  
                counter += 1  
                time.sleep(incremental_backoff)  
        except Exception as ex:  
            counter += 1  
            print('Error - Retry count:', counter, ex)  
            return None  
  
# Function to get SQL Server schema information  
def get_sql_server_schema_info(server, database, username, password):  
    conn = pyodbc.connect(  
        f'DRIVER={{ODBC Driver 18 for SQL Server}};'  
        f'SERVER={server};'  
        f'DATABASE={database};'  
        f'UID={username};'  
        f'PWD={password}'  
    )  
    cursor = conn.cursor()  
  
    # Fetch all table names  
    cursor.execute("SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE';")  
    tables = cursor.fetchall()  
  
    schema_info = {}  
    for table in tables:  
        table_owner = table[0]  
        table_name = table[1]  
  
        # Fetch column details  
        cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{table_name}';")  
        columns = cursor.fetchall()  
  
        # Fetch primary key details  
        cursor.execute(f"""  
            SELECT COLUMN_NAME  
            FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS TC  
            JOIN INFORMATION_SCHEMA.CONSTRAINT_COLUMN_USAGE CCU   
            ON TC.CONSTRAINT_NAME = CCU.CONSTRAINT_NAME  
            WHERE TC.TABLE_NAME = '{table_name}' AND TC.CONSTRAINT_TYPE = 'PRIMARY KEY';  
        """)  
        primary_keys = cursor.fetchall()  
        primary_keys = [pk[0] for pk in primary_keys]  
  
        # Fetch foreign key details  
        cursor.execute(f"""  
            SELECT   
                KCU.COLUMN_NAME,  
                KCU.CONSTRAINT_NAME,  
                RC.UPDATE_RULE,  
                RC.DELETE_RULE,  
                KCU2.TABLE_NAME AS REFERENCED_TABLE_NAME,  
                KCU2.COLUMN_NAME AS REFERENCED_COLUMN_NAME  
            FROM   
                INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS RC  
                JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE KCU  
                ON KCU.CONSTRAINT_NAME = RC.CONSTRAINT_NAME  
                JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE KCU2  
                ON KCU2.CONSTRAINT_NAME = RC.UNIQUE_CONSTRAINT_NAME  
            WHERE KCU.TABLE_NAME = '{table_name}';  
        """)  
        foreign_keys = cursor.fetchall()  
  
        column_info = []  
        for col in columns:  
            column_name = col[0]  
            column_type = col[1]  
            description = f"{column_name} ({column_type})"  
            if column_name in primary_keys:  
                description += " [Primary Key]"  
            for fk in foreign_keys:  
                if column_name == fk[0]:  
                    description += f" [Foreign Key to {fk[4]}({fk[5]})]"  
            column_info.append((column_name, column_type, description))  
  
        # Fetch sample rows  
        markdown_table = ''  
        try:  
            cursor.execute(f"SELECT TOP 40 * FROM {table_owner}.{table_name}")  
            rows = cursor.fetchall()  
  
            # Randomly select 5-10 rows for diversity  
            selected_rows = random.sample(rows, min(len(rows), 3))  
  
            # Convert sample rows to markdown table format  
            if selected_rows:  
                headers = [desc[0] for desc in cursor.description]  
                markdown_table += "\n| " + " | ".join(headers) + " |\n"  
                markdown_table += "| " + " | ".join(['---'] * len(headers)) + " |\n"  
                for row in selected_rows:  
                    markdown_table += "| " + " | ".join([str(cell) if cell is not None else 'NULL' for cell in row]) + " |\n"  
  
        except pyodbc.ProgrammingError as e:  
            if 'ODBC SQL type -151' in str(e):  
                markdown_table += "\n| Error | This table contains unsupported data types and cannot be sampled |\n"  
            else:  
                raise e  
  
        column_info.append(('Sample Data from Table', markdown_table, ''))  
        schema_info[table_owner + '.' + table_name] = column_info  
  
    conn.close()  
    return schema_info  
  
# Function to execute SQL query  
def execute_query(query):  
    conn = pyodbc.connect(  
        f'DRIVER={{ODBC Driver 18 for SQL Server}};'  
        f'SERVER={server};'  
        f'DATABASE={database};'  
        f'UID={username};'  
        f'PWD={password}'  
    )  
  
    df = pd.read_sql(query, conn)  
  
    # Convert dataframe to Markdown table  
    markdown_table = df.to_markdown(index=False)  
  
    return markdown_table  

# Function to generate table description  
def generate_table_description(table_name, columns):  
    schema_details = f"Table Name: {table_name}\nColumns:\n"  
    for column_name, column_type, description in columns:  
        if column_name == 'Sample Data from Table':  
            schema_details += f"- {column_name} \n {column_type}\n"  
        else:  
            schema_details += f"- {column_name} ({column_type}): {description}\n"  
    return generate_oai_table_desc(schema_details, openai_gpt_api_base, openai_gpt_api_key, openai_gpt_api_version, openai_gpt_model)  
  
# Function to vectorize descriptions  
def vectorize_descriptions(table_descriptions):  
    description_vectors = {}  
    for table_name, description in table_descriptions.items():  
        description_vectors[table_name] = generate_embedding(description, openai_embedding_api_version, openai_embedding_api_base, openai_embedding_api_key, openai_embedding_model)  
    return description_vectors  

# Function to execute query request  
def execute_query_request(question, merged_table_descriptions_markdown):  
    sql_query_response = generate_query(question, merged_table_descriptions_markdown, None, openai_gpt_api_base, openai_gpt_api_key, openai_gpt_api_version, openai_gpt_model)  
    if '```sql' in sql_query_response:  
        sql_query_response = sql_query_response[sql_query_response.find('```sql') + 6:]  
    if '```' in sql_query_response:  
        sql_query_response = sql_query_response[:sql_query_response.find('```')]  
      
    print(sql_query_response)  
      
    response = execute_query(sql_query_response)  
    answer = generate_text_answer(question, response, openai_gpt_api_base, openai_gpt_api_key, openai_gpt_api_version, openai_gpt_model)  
      
    display(Markdown('## ' + question + '\n\n' + response))  
    print('\n --------------------------------------- \n')  
    print('Textual Answer')  
    print(answer)  
  



In [15]:
# Generate schema information  
print ('Retrieving SQL Schema information...')
schema_info = get_sql_server_schema_info(server, database, username, password)  
  
# Generate descriptions for all tables  
print ('Creating table descriptions...')
table_descriptions = {}  
for table_name, columns in schema_info.items():  
    print ('Processing table:', table_name)
    description = generate_table_description(table_name, columns)  
    table_descriptions[table_name] = description  
  
# Save table descriptions to a file  
with open('table_descriptions.pkl', 'wb') as pkl_out:  
    pickle.dump(table_descriptions, pkl_out)  
  
# # Vectorize descriptions  
# print ('Vectorizing table details...')
# description_vectors = vectorize_descriptions(table_descriptions)  
  
# # Save description vectors to a file  
# with open('description_vectors.pkl', 'wb') as pkl_out:  
#     pickle.dump(description_vectors, pkl_out)  
  
# Merge table descriptions into a single markdown string  
print ('Merging table descriptions into markdown...')
merged_table_descriptions_markdown = ''  
for td in table_descriptions:  
    merged_table_descriptions_markdown += table_descriptions[td] + '\n\n'  
  
# Save the merged table descriptions to a text file  
with open('merged_table_descriptions_markdown.txt', 'w') as f_out:  
    f_out.write(merged_table_descriptions_markdown)  
  
print('Markdown length:', len(merged_table_descriptions_markdown))
  


Retrieving SQL Schema information...
Creating table descriptions...
Processing table: dbo.InventoryItems ...
Processing table: dbo.CustomerAccounts ...
Processing table: dbo.BankInformation ...
Processing table: dbo.ExpenseClaimAudit ...
Processing table: dbo.BudgetAllocationAudit ...
Processing table: dbo.TaxInformation ...
Processing table: dbo.PortfolioInvestments ...
Processing table: dbo.FinancialStatementAudit ...
Processing table: dbo.FinancialRiskAudit ...
Processing table: dbo.FinancialAuditMaster ...
Processing table: dbo.EmployeeInfo ...
Processing table: dbo.LoanRepayments ...
Processing table: dbo.sysdiagrams ...
Processing table: dbo.ProjectTasks ...
Processing table: dbo.VendorDetails ...
Merging table descriptions into markdown...
Markdown length: 37722


In [16]:
# Read the merged table descriptions from the text file  
with open('merged_table_descriptions_markdown.txt', 'r') as f_in:  
    merged_table_descriptions_markdown = f_in.read()  


In [20]:
# Example usage  
question = "What is the most common loan type customers have?"  
execute_query_request(question, merged_table_descriptions_markdown)  



SELECT TOP 1
    LoanType,
    COUNT(*) AS LoanTypeCount
FROM
    dbo.LoanRepayments
GROUP BY
    LoanType
ORDER BY
    LoanTypeCount DESC;



/tmp/ipykernel_179/242020169.py:362: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## What is the most common loan type customers have?

| LoanType              |   LoanTypeCount |
|:----------------------|----------------:|
| Audit Compliance Loan |            7958 |


 --------------------------------------- 

Textual Answer
The most common loan type customers have is the Audit Compliance Loan, with a count of 7,958.
